In [1]:
import os # 경로를 지정하거나 연산에 쓰기 위하여
import re # regular expression 주어진 규칙에 맞는 언어 연산
import pandas as pd # 데이터처리 및 데이터과학을 위한 라이브러리
import tensorflow as tf # 데이터 다운받는 용도로 필요
from tensorflow.keras import utils # 데이터 다운받는 용도로 필요

# IMBD 데이터 다운로드
data_set = tf.keras.utils.get_file(
    fname = "imdb.tar.gz", # 다운받은 파일의 이름
    origin = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",
    extract = True)
def directory_data(directory):
  data = {}
  data["review"] = []
  for file_path in os.listdir(directory):
    with open(os.path.join(directory, file_path), "r", encoding='utf-8') as file:
      data["review"].append(file.read())
  return pd.DataFrame.from_dict(data)
def data(directory):
  pos_df = directory_data(os.path.join(directory, "pos"))
  neg_df = directory_data(os.path.join(directory, "neg"))
  pos_df["sentiment"] = 1
  neg_df["sentiment"] = 0
  return pd.concat([pos_df, neg_df])
train_df = data(os.path.join(os.path.dirname(data_set), "aclImdb", "train"))
test_df = data(os.path.join(os.path.dirname(data_set), "aclImdb", "test"))
imdb_pd = pd.concat([train_df, test_df])
imdb_pd.head()

84125825/84125825 [==============================] - 4s 0us/step


,review,sentiment
0,The morbid Catholic writer Gerard Reve (Jeroen...,1
1,"""Kolchak: the Night Stalker"" is a hugely enter...",1
2,"Like the great classic Bugs Bunny cartoons, th...",1
3,A wonderful early musical film from Rene Clair...,1
4,"This is a good movie, a good family movie to w...",1


In [5]:
from bs4 import BeautifulSoup

In [6]:
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')
set_stopwords = set(stopwords.words('english')) # 영어 불용어를 가져와 집합으로 구성

def preprocessing(review, remove_stopwords=True):
    review_text = BeautifulSoup(review, 'html5lib').get_text()
    review_text = re.sub("[^a-zA-Z]", " ", review_text) # subtitution 즉 대체하라는 의미 -> a-z, A-Z를 제외함 (^a-zA-Z), 알파벳에 해당하지 않는 문자는 빈 칸으로 대체

    if remove_stopwords:
        words = review_text.split()
        words = [w for w in words if not w in set_stopwords]
        review_text = ' '.join(words)
    review_text = review_text.lower()
    return review_text



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
list_reviews = list(imdb_pd['review'])
# print(list_reviews[0])
# print(preprocessing(list_reviews[0]))

list_preprocessing_reviews = []

for review in list_reviews:
    list_preprocessing_reviews.append(preprocessing(review))

list_preprocessing_data = pd.DataFrame({'review': list_preprocessing_reviews, 'sentiment': imdb_pd['sentiment']})
list_reviews = list(list_preprocessing_data['review'])
list_sentiments = list(list_preprocessing_data['sentiment'])
print(list_reviews[0])

<ipython-input-6-7b75737a80f1>:8: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(review, 'html5lib').get_text()


the morbid catholic writer gerard reve jeroen krabb homosexual alcoholic frequent visions death invited give lecture literature club vlissingen while railway station amsterdam feels non corresponded attraction handsome man embarks another train gerard introduced treasurer club beautician christine halsslag ren e soutendijk wealthy widow owns beauty shop sphinx one night stand on next morning gerard sees picture christine boyfriend herman thom hoffman recognizes man saw train station he suggests bring herman house spend couple days together secret intention seducing man christine travels k ln bring boyfriend gerard stays alone house he drinks whiskey snoops safe finding three film reels names men decides watch footages discover christine married three guys died tragic accidents later gerard believes christine witch question whether herman doomed fourth husband the ambiguous the vierde man another magnificent feature paul verhoeven dutch phase the story supported excellent screenplay use

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import numpy as np

vector = TfidfVectorizer(max_features=5000) # 주어진 문장을 벡터로 만드는 객체 생성
x = vector.fit_transform(list_reviews).toarray()
y = np.array(list_sentiments)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print(x.shape)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

import torch

x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(y_train)
W = torch.randn([5000,1], requires_grad=True)
b = torch.randn([1], requires_grad=True)

optimizer = torch.optim.SGD([W,b], lr=0.01)

# 딥러닝의 1단계: 모델을 만듦
def H(x):
    return torch.sigmoid(torch.matmul(x, W) + b) # H(x) = Wx + b

# 딥러닝의 2단계: 학습을 한다. epochs=2000
for step in range(2000):
    cost = -torch.mean(y_train*torch.log(H(x_train)) + (1-y_train)*torch.log(1-H(x_train))) # cost = 평균{(H(x) - y)의 제곱}
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

# 딥러닝 3단계: 추론/테스트을 한다
x_test = torch.FloatTensor(x_test[0])

print(H(x_test))

(50000, 5000)
(40000, 5000)
(10000, 5000)
(40000,)
(10000,)


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

vector = TfidfVectorizer(max_features=5000) # 주어진 문장을 벡터로 만드는 객체 생성
x = vector.fit_transform(list_reviews).toarray()
y = np.array(list_sentiments)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print(x.shape)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

# Model
model = Sequential()
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

# Training
model.fit(x_train, y_train, epochs=50)

# Testing
_, accuracy = model.evaluate(x_test, y_test)
print('Accuracy: ', accuracy)
model.summary()

(50000, 5000)
(40000, 5000)
(10000, 5000)
(40000,)
(10000,)
Epoch 1/50
1250/1250 [==============================] - 3s 2ms/step - loss: 0.6905 - accuracy: 0.5799
Epoch 2/50
1250/1250 [==============================] - 2s 2ms/step - loss: 0.6856 - accuracy: 0.7122
Epoch 3/50
1250/1250 [==============================] - 2s 2ms/step - loss: 0.6807 - accuracy: 0.7620
Epoch 4/50
1250/1250 [==============================] - 2s 2ms/step - loss: 0.6760 - accuracy: 0.7853
Epoch 5/50
1250/1250 [==============================] - 3s 2ms/step - loss: 0.6713 - accuracy: 0.8051
Epoch 6/50
1250/1250 [==============================] - 3s 2ms/step - loss: 0.6668 - accuracy: 0.8146
Epoch 7/50
1250/1250 [==============================] - 2s 2ms/step - loss: 0.6624 - accuracy: 0.8177
Epoch 8/50
1250/1250 [==============================] - 2s 2ms/step - loss: 0.6580 - accuracy: 0.8188
Epoch 9/50
1250/1250 [==============================] - 2s 2ms/step - loss: 0.6538 - accuracy: 0.8222
Epoch 10/50
1250/1250 

In [41]:
query = [input()]
target = vector.transform(query).toarray()
y_pred = model.predict(target)
pred = np.where(y_pred > 0.5, 1, 0)

if pred == 0:
    print('Bad Review')
elif pred == 1:
    print('Good Review')

이 영화는 좋지도 나쁘지도 않은 그럭저럭한 영화입니다. 재밌긴 한데 중간에 지루해서 살짝 졸았어요. 그치만 배우들의 연기력과 외모가 너무 출중했습니다.
1/1 [==============================] - 0s 32ms/step
Good Review
